# 1. Rysowanie Dłoni

Rozpoznawanie dłonie polega na wyznaczeniu pozycji elementów charakterystycznych dłoni. W sumie można wyznaczych ich 21. Są to między innymi stawy, nadgarstek lub końcówki palców. Współrzędne są obliczne względem położenia nagarstka.  

<img src=https://i.imgur.com/qpRACer.png />

Rozpoczynamy od zaimportowania odpowiednich bibliotek.

OpenCV pozowli na przeprowadzenie wstępnych przekształceń obrazu, w taki sposób, aby biblioteka MediaPipe mogła poprawnie rozpoznać dłoń oraz jej elementy charaktterystyczne. 

In [3]:
import mediapipe as mp
import cv2
import numpy as np

Wybieramy dwa obiekty klasy mp.solutions:

1. mp_drawing - pozowli na naniesienie punktów na elementy charakterystyczne dłoni oraz linii ich łączących. 
2. mp_hands - zostanie wykorzystany do rozpoznania dłoni z wybraną dokładnością. 

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

Wstępne ropoznanie dłoni i nanisieni grafiki na obraz pobrany z kamery.

# 2. Zapis pozycji elementów charakterystycznych do pliku CSV

In [5]:
import csv
import os
import numpy as np

Tworzymy oznaczenia kolumn (klasy, współrzędne)

In [6]:
landmarks = ['class']
for val in range(1, 21):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [ ]:
landmarks

Tworzymy plik CSV i zapisujemy do niego oznaczenia kolumn.

In [7]:
FILE_NAME='wrist_normalized_positions.csv'

In [8]:
with open(FILE_NAME, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

<img src=https://pastevents.impactcee.com/wp-content/uploads/2016/10/DayTranslationsBlog-Learn-American-Sign-Language.jpg />

In [71]:
class_name="K"

In [72]:
cap = cv2.VideoCapture(0)

detections = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
        try:
            hand_landmarks = results.multi_hand_landmarks[0].landmark
            wrist = hand_landmarks[0]
                
            hand_landmarks_row = np.zeros((20,3))
            for i in range(1, len(hand_landmarks)):
                hand_landmarks_row[i-1]=[hand_landmarks[i].x-wrist.x, hand_landmarks[i].y-wrist.y, hand_landmarks[i].z-wrist.z]
                
            print(hand_landmarks_row)
            hand_landmarks_row = hand_landmarks_row.flatten()
            hand_landmarks_row = list(hand_landmarks_row/np.max(np.absolute(hand_landmarks_row)))
            
            hand_landmarks_row.insert(0, class_name)
            
            
            with open(FILE_NAME, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(hand_landmarks_row)
                detections += 1
        except:
            pass
        
        if detections == 500:
            break

        #image = cv2.flip(image, 0)
        cv2.imshow("Gesture Training", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[[-0.02850538 -0.04862732 -0.03714035]
 [-0.01980788 -0.10939908 -0.05494722]
 [ 0.02540034 -0.10842663 -0.06960775]
 [ 0.06401646 -0.0708335  -0.07913626]
 [ 0.03751552 -0.1891951  -0.03750172]
 [ 0.04567111 -0.1726734  -0.06247746]
 [ 0.0245958  -0.11573619 -0.06571057]
 [ 0.01396519 -0.09527582 -0.06528076]
 [ 0.0737735  -0.15837252 -0.03475315]
 [ 0.08281338 -0.14379358 -0.06364786]
 [ 0.05135554 -0.0957551  -0.05953335]
 [ 0.03786683 -0.08623052 -0.05084747]
 [ 0.09829515 -0.11947942 -0.0373826 ]
 [ 0.10799575 -0.11195534 -0.06569904]
 [ 0.07388222 -0.07214922 -0.05259486]
 [ 0.05546325 -0.06032109 -0.03672246]
 [ 0.11756933 -0.07711112 -0.04472195]
 [ 0.16132528 -0.10135722 -0.06649622]
 [ 0.18641853 -0.11192536 -0.0663587 ]
 [ 0.21153957 -0.13572538 -0.05971418]]
[[-0.01559448 -0.04804838 -0.02759765]
 [-0.00087059 -0.10922623 -0.04457327]
 [ 0.03574413 -0.11749476 -0.05976363]
 [ 0.06714094 -0.08412504 -0.07069714]
 [ 0.04529124 -0.1700294  -0.03157885]
 [ 0.0708425  -0.1982960

In [13]:
len(hand_landmarks_row)

61

# 3. Trening modeli z wykorzystaniem Scikit Learn

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [40]:
df = pd.read_csv(FILE_NAME)

In [41]:
df.head()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,A,-0.285040,0.002379,-0.156813,-0.599328,-0.291157,-0.268524,-0.701899,-0.647329,-0.377960,...,-0.365519,0.161154,-0.750611,-0.480271,0.173212,-0.495469,-0.437673,0.231874,-0.473569,-0.369932
1,A,-0.294122,0.059555,-0.220862,-0.639607,-0.296191,-0.308676,-0.769940,-0.719772,-0.369607,...,-0.068271,0.050391,-0.996935,-0.156264,0.072934,-0.712266,-0.130468,0.122912,-0.642172,-0.073136
2,A,-0.295894,-0.018408,-0.093793,-0.596678,-0.257949,-0.141282,-0.706606,-0.619757,-0.197386,...,-0.088853,0.029423,-1.000000,-0.137068,0.045982,-0.777570,-0.094996,0.087181,-0.659501,-0.034887
3,A,-0.303750,-0.000960,-0.103575,-0.612420,-0.233723,-0.161898,-0.742600,-0.595462,-0.225911,...,-0.088627,-0.014992,-1.000000,-0.128778,0.009571,-0.777341,-0.082763,0.058416,-0.686582,-0.022444
4,A,-0.304287,-0.022814,-0.082761,-0.597325,-0.237812,-0.118089,-0.727407,-0.588854,-0.161504,...,-0.059354,-0.010492,-1.000000,-0.090168,0.011996,-0.784827,-0.044144,0.052934,-0.669614,0.015007


In [42]:
df.tail()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
2995,F,-0.110935,-0.136048,-0.006784,-0.189454,-0.313123,-0.013506,-0.232303,-0.464713,-0.030220,...,-0.104070,0.080098,-0.614573,-0.130655,0.066882,-0.727747,-0.140805,0.049339,-0.830891,-0.146760
2996,F,-0.102987,-0.159188,0.005632,-0.168967,-0.347602,0.000879,-0.205299,-0.495728,-0.016582,...,-0.115192,0.124395,-0.608948,-0.141779,0.117200,-0.726063,-0.150234,0.102729,-0.829137,-0.155300
2997,F,-0.090473,-0.175105,0.011224,-0.141029,-0.366309,0.007791,-0.170135,-0.514907,-0.009599,...,-0.120755,0.167764,-0.584002,-0.146825,0.170863,-0.701141,-0.155722,0.167093,-0.806599,-0.161929
2998,F,-0.081191,-0.181998,0.006470,-0.121062,-0.381338,0.000430,-0.144098,-0.532259,-0.019003,...,-0.124744,0.192429,-0.564765,-0.151962,0.202200,-0.682109,-0.160416,0.206499,-0.793444,-0.166041
2999,F,-0.078902,-0.180341,0.002367,-0.114171,-0.381713,-0.002101,-0.134015,-0.534663,-0.018752,...,-0.111868,0.204269,-0.558054,-0.136950,0.213468,-0.676189,-0.144692,0.216562,-0.790213,-0.149760


In [45]:
x = df.drop('class', axis=1)
y = df['class']

In [46]:
x

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,-0.285040,0.002379,-0.156813,-0.599328,-0.291157,-0.268524,-0.701899,-0.647329,-0.377960,-0.574921,...,-0.365519,0.161154,-0.750611,-0.480271,0.173212,-0.495469,-0.437673,0.231874,-0.473569,-0.369932
1,-0.294122,0.059555,-0.220862,-0.639607,-0.296191,-0.308676,-0.769940,-0.719772,-0.369607,-0.636165,...,-0.068271,0.050391,-0.996935,-0.156264,0.072934,-0.712266,-0.130468,0.122912,-0.642172,-0.073136
2,-0.295894,-0.018408,-0.093793,-0.596678,-0.257949,-0.141282,-0.706606,-0.619757,-0.197386,-0.588880,...,-0.088853,0.029423,-1.000000,-0.137068,0.045982,-0.777570,-0.094996,0.087181,-0.659501,-0.034887
3,-0.303750,-0.000960,-0.103575,-0.612420,-0.233723,-0.161898,-0.742600,-0.595462,-0.225911,-0.631855,...,-0.088627,-0.014992,-1.000000,-0.128778,0.009571,-0.777341,-0.082763,0.058416,-0.686582,-0.022444
4,-0.304287,-0.022814,-0.082761,-0.597325,-0.237812,-0.118089,-0.727407,-0.588854,-0.161504,-0.639897,...,-0.059354,-0.010492,-1.000000,-0.090168,0.011996,-0.784827,-0.044144,0.052934,-0.669614,0.015007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-0.110935,-0.136048,-0.006784,-0.189454,-0.313123,-0.013506,-0.232303,-0.464713,-0.030220,-0.245584,...,-0.104070,0.080098,-0.614573,-0.130655,0.066882,-0.727747,-0.140805,0.049339,-0.830891,-0.146760
2996,-0.102987,-0.159188,0.005632,-0.168967,-0.347602,0.000879,-0.205299,-0.495728,-0.016582,-0.216444,...,-0.115192,0.124395,-0.608948,-0.141779,0.117200,-0.726063,-0.150234,0.102729,-0.829137,-0.155300
2997,-0.090473,-0.175105,0.011224,-0.141029,-0.366309,0.007791,-0.170135,-0.514907,-0.009599,-0.180931,...,-0.120755,0.167764,-0.584002,-0.146825,0.170863,-0.701141,-0.155722,0.167093,-0.806599,-0.161929
2998,-0.081191,-0.181998,0.006470,-0.121062,-0.381338,0.000430,-0.144098,-0.532259,-0.019003,-0.152823,...,-0.124744,0.192429,-0.564765,-0.151962,0.202200,-0.682109,-0.160416,0.206499,-0.793444,-0.166041


In [47]:
y

0       A
1       A
2       A
3       A
4       A
       ..
2995    F
2996    F
2997    F
2998    F
2999    F
Name: class, Length: 3000, dtype: object

Wszystkie pobrane dane dzielimy na dwie części, pierwsza posłuży do trenowania, druga do testwowania.

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3451)

In [49]:
type(y_train)

pandas.core.series.Series

In [50]:
x_train

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
424,-0.281196,-0.015961,-0.128701,-0.577204,-0.383838,-0.195233,-0.691914,-0.763726,-0.256833,-0.655827,...,-0.116979,0.050096,-0.789293,-0.170955,0.045597,-0.534294,-0.144605,0.023405,-0.542543,-0.100232
1541,-0.290710,-0.057261,0.097534,-0.490595,-0.238933,0.050701,-0.656194,-0.316764,-0.044762,-0.783618,...,-0.483136,-0.301124,-0.782634,-0.617775,-0.482284,-0.740717,-0.632576,-0.594843,-0.651356,-0.627023
1241,-0.173587,-0.121465,0.060632,-0.300337,-0.294862,0.062389,-0.416462,-0.395026,0.031207,-0.514963,...,-0.157852,-0.037306,-0.797043,-0.198337,-0.125596,-0.878883,-0.209765,-0.216222,-0.910022,-0.217237
1341,-0.205419,-0.041300,0.027810,-0.379514,-0.167100,0.017098,-0.519793,-0.232836,-0.021912,-0.616110,...,-0.126591,-0.119717,-0.837891,-0.162245,-0.213383,-0.925693,-0.175876,-0.305578,-0.954419,-0.183705
1791,-0.170188,-0.094514,0.034595,-0.307901,-0.182195,-0.007732,-0.415057,-0.223982,-0.069329,-0.450233,...,-0.209232,-0.290612,-0.404043,-0.235312,-0.360915,-0.348201,-0.202589,-0.394770,-0.272970,-0.174946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,-0.134680,-0.075216,-0.025160,-0.252452,-0.224242,-0.033054,-0.313804,-0.378832,-0.045652,-0.309344,...,-0.068271,0.042676,-0.628064,-0.086915,0.036193,-0.743333,-0.095826,0.024621,-0.844673,-0.101568
97,-0.220501,-0.150354,-0.036033,-0.362431,-0.479376,-0.091482,-0.397095,-0.765488,-0.157343,-0.396446,...,-0.224848,0.080360,-0.571409,-0.270819,0.058561,-0.382178,-0.247236,0.109749,-0.368696,-0.217753
2198,-0.207451,-0.079085,-0.091905,-0.342052,-0.325813,-0.132593,-0.260833,-0.570174,-0.181529,-0.080361,...,-0.091444,0.008471,-0.896671,-0.134623,-0.009954,-0.811349,-0.129961,-0.014013,-0.674685,-0.105513
1678,-0.182856,-0.078552,0.041141,-0.305452,-0.173836,0.013985,-0.408963,-0.225724,-0.034531,-0.451987,...,-0.168717,-0.278786,-0.462768,-0.186998,-0.354952,-0.399271,-0.164978,-0.390514,-0.317186,-0.145855


# 4. Trenowanie Klasyfikujących Modeli Uczenia Maszynowego

In [51]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

Tworzymy słownik przechowywujący 4 metody uczenie maszynowego wraz z metodą normalizacji.

In [52]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rd':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [53]:
y_train

424     A
1541    D
1241    C
1341    C
1791    D
       ..
2504    F
97      A
2198    E
1678    D
2143    E
Name: class, Length: 2100, dtype: object

Trenujemy 4 różne modele jednocześnie. 

!!! PRZETESTOWAĆ INNE METODY !!!

In [54]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [64]:
fit_models['rf'].predict(x_test)

array(['open', 'open', 'open', 'fist', 'open', 'open', 'open', 'fist',
       'open', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'fist',
       'fist', 'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist', 'open',
       'open', 'open', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'open', 'fist', 'fist', 'open',
       'fist', 'fist', 'open', 'open', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'open',
       'fist', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'fist', 'open', 'fist', 'fist', 'fist',
       'fist', 'open', 'fist', 'open', 'open', 'open', 'open', 'fist',
       'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'open', 'open', 'open', 'open', 'fist',
      

# 5. Ewaluacja Modelu

In [55]:
from sklearn.metrics import accuracy_score
import pickle

Porównujemy dokładność każdego modelu wykorzystując funkcję accuracy_score

In [56]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9977777777777778
rd 0.9933333333333333
rf 0.9966666666666667
gb 0.9922222222222222


In [57]:
with open('sign_language_alphabet.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

# 5. Detekcje

Powtórnie ładujemy model.

In [58]:
with open('sign_language_alphabet.pkl', 'rb') as f:
    model = pickle.load(f)

In [59]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [50]:
model.predict(x_test)

array(['open', 'open', 'open', 'fist', 'open', 'open', 'open', 'fist',
       'open', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'fist',
       'fist', 'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'open', 'fist', 'open', 'fist', 'open', 'fist', 'open',
       'open', 'fist', 'fist', 'fist', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'open', 'fist', 'fist', 'open',
       'fist', 'fist', 'open', 'open', 'fist', 'fist', 'open', 'fist',
       'fist', 'fist', 'open', 'open', 'fist', 'open', 'fist', 'open',
       'fist', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'fist', 'open', 'fist', 'fist', 'fist',
       'fist', 'open', 'fist', 'open', 'open', 'open', 'open', 'fist',
       'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open',
       'open', 'fist', 'open', 'open', 'open', 'open', 'open', 'fist',
      

In [51]:
pd.DataFrame(x_test)

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
311,-0.122366,0.154571,-0.031917,-0.199848,0.348932,-0.042279,-0.261205,0.517757,-0.050960,-0.327744,...,-0.003933,0.232572,0.484272,-0.016964,0.259329,0.536242,-0.026011,0.283341,0.584262,-0.032395
472,-0.120036,-0.118734,-0.029220,-0.216168,-0.285095,-0.058979,-0.286563,-0.416182,-0.097176,-0.357530,...,-0.132790,0.215612,-0.546907,-0.171076,0.242087,-0.638599,-0.188892,0.254576,-0.727021,-0.201592
387,-0.018149,-0.325881,-0.009420,0.062272,-0.614014,-0.007430,0.150967,-0.824606,-0.017904,0.192865,...,-0.034878,0.672843,-0.164905,-0.062415,0.775462,-0.187457,-0.075748,0.870415,-0.223513,-0.079016
560,-0.289820,0.108541,-0.205375,-0.734906,-0.080346,-0.264466,-0.946463,-0.400162,-0.294812,-0.861793,...,0.026708,-0.162364,-1.000000,-0.044242,-0.080222,-0.733392,-0.014111,-0.096771,-0.623113,0.046288
423,-0.117827,-0.138752,-0.033318,-0.211735,-0.323980,-0.066010,-0.290154,-0.462984,-0.106554,-0.373721,...,-0.126488,0.336963,-0.495040,-0.167951,0.411554,-0.564842,-0.191383,0.474222,-0.633846,-0.208356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,0.106971,-0.360454,0.092596,0.081788,-0.660642,0.103109,-0.037583,-0.805570,0.092885,-0.151875,...,-0.126584,-0.328395,-0.709786,-0.071641,-0.302990,-0.609321,-0.037710,-0.262762,-0.541080,-0.025406
181,0.210523,0.083584,-0.035197,0.428733,0.052988,-0.064152,0.603579,0.009858,-0.095681,0.747452,...,-0.142963,0.289274,-0.735975,-0.197838,0.332259,-0.869786,-0.231096,0.372779,-0.984128,-0.254844
896,-0.284715,-0.232632,0.005019,-0.449822,-0.521568,-0.017280,-0.466985,-0.781971,-0.079262,-0.379222,...,-0.252696,0.082159,-0.875625,-0.313326,0.076459,-0.670760,-0.265768,0.146113,-0.561699,-0.207085
637,-0.300201,0.012477,-0.129525,-0.658218,-0.200261,-0.237174,-0.849936,-0.453128,-0.338899,-0.803452,...,-0.243938,-0.012868,-0.759016,-0.292509,0.006543,-0.533807,-0.265404,-0.017603,-0.560098,-0.232358


In [61]:
cap = cv2.VideoCapture(0)

detections = 0
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
        try:
            hand_landmarks = results.multi_hand_landmarks[0].landmark
            wrist = hand_landmarks[0]
                
            hand_landmarks_row = np.zeros((20,3))
            for i in range(1, len(hand_landmarks)):
                hand_landmarks_row[i-1]=[hand_landmarks[i].x-wrist.x, hand_landmarks[i].y-wrist.y, hand_landmarks[i].z-wrist.z]
                
            # print(hand_landmarks_row)
            hand_landmarks_row = hand_landmarks_row.flatten()
            hand_landmarks_row = list(hand_landmarks_row/np.max(np.absolute(hand_landmarks_row)))
            
            #Make Detections
            x = pd.DataFrame([hand_landmarks_row])

            #gesture_prob = model.predict_proba(x)[0]

            with warnings.catch_warnings():
               warnings.filterwarnings("ignore")
            print(model.predict(x))
            cv2.putText(image, gesture_class, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        except:
            pass
        

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    print(image.shape)

    cap.release()
    cv2.destroyAllWindows()

(480, 640, 3)


In [ ]:
hand_landmarks_row